In [13]:
import geopandas as gpd
import pandas as pd
from collections import Counter
import numpy as np

# ===== 設定參數 =====
LIQUEFACTION_PATH = 'TPLiquid_84.GEOJSON'  # 液化潛勢資料
STAT_AREA_PATH = 'STAT/113年12月臺北市統計區人口統計_最小統計區/tp_最小統計區.geojson'  # 最小統計區
OUTPUT_PATH = 'taipei_liquefaction_risk.geojson'
OVERLAP_THRESHOLD = 0.5

# 液化等級對應數值 (根據你的資料調整)
RISK_VALUES = {
    '1':0.67,
    '2': 0.33, 
    '3': 0.08,
    '4': 0
}

# '低潛勢區': 2.5/30 ≈ 0.08,   # PL中位數 2.5
# '中潛勢區': 10/30 ≈ 0.33,    # PL中位數 10
# '高潛勢區': 20/30 ≈ 0.67     # PL假設值 20

print("=" * 60)
print("統計區液化風險值計算 (面積加權平均法)")
print("=" * 60)

# ===== Step 1: 讀取資料 =====
print("\n[1/4] 讀取資料...")
liquefaction = gpd.read_file(LIQUEFACTION_PATH)
stat_areas = gpd.read_file(STAT_AREA_PATH)

print(f"  ✓ 液化潛勢網格: {len(liquefaction)} 筆")
print(f"  ✓ 最小統計區: {len(stat_areas)} 筆")

# 自動偵測液化等級欄位
risk_column = None
for col in liquefaction.columns:
    if 'class' in col or 'risk' in col.lower() or '等級' in col:
        risk_column = col
        break

if risk_column is None:
    print("\n  可用欄位:", liquefaction.columns.tolist())
    risk_column = input("  請輸入液化等級欄位名稱: ")

print(f"  使用欄位: {risk_column}")
print(f"  液化等級: {liquefaction[risk_column].unique()}")

# ===== Step 2: 座標系統統一 =====
print("\n[2/4] 座標系統處理...")

# 統一座標系統
if liquefaction.crs != stat_areas.crs:
    liquefaction = liquefaction.to_crs(stat_areas.crs)

# 轉換至 wgs84 計算面積
stat_areas_meter = stat_areas.to_crs('EPSG:4326')
liquefaction_meter = liquefaction.to_crs('EPSG:4326')

print(f"  ✓ 已統一座標系統")

# ===== Step 3: 空間疊合與計算 =====
print("\n[3/4] 計算風險值...")

# 空間疊合
overlay = gpd.overlay(
    stat_areas_meter,
    liquefaction_meter,
    how='intersection'
)

# 計算交集面積
overlay['intersect_area'] = overlay.geometry.area

# 計算每個統計區的加權平均風險值
risk_values = []

for stat_id in stat_areas_meter['U_ID'].unique():
    stat_overlay = overlay[overlay['U_ID'] == stat_id]
    
    if len(stat_overlay) == 0:
        # 無液化資料
        risk_values.append({
            'U_ID': stat_id,
            'liq_risk': 0
        })
        continue

    # 計算統計區總面積
    stat_area_total = stat_areas_meter[
        stat_areas_meter['U_ID'] == stat_id
    ].geometry.area.iloc[0]
    
    # 計算液化區覆蓋該統計區的面積
    overlap_area = stat_overlay['intersect_area'].sum()
    
    # 計算覆蓋率
    overlap_ratio = overlap_area / stat_area_total
    
    # === 覆蓋率門檻判斷 ===
    if overlap_ratio < OVERLAP_THRESHOLD:
        # 覆蓋率不足,視為邊緣效應,風險設為 0
        risk_values.append({
            'U_ID': stat_id,
            'liq_risk': 0,
            'overlap_ratio': round(overlap_ratio, 3)
        })
        continue

    # 計算總面積
    total_area = stat_overlay['intersect_area'].sum()
    
    # 面積加權平均
    weighted_sum = 0
    for _, row in stat_overlay.iterrows():
        risk_val = RISK_VALUES.get(row[risk_column], 0)
        weight = row['intersect_area'] / total_area
        weighted_sum += risk_val * weight
    
    # 標準化到 0-1
    risk_normalized = weighted_sum 
    
    risk_values.append({
        'U_ID': stat_id,
        'liq_risk': round(risk_normalized, 3)
    })

# 轉換為 DataFrame
risk_df = pd.DataFrame(risk_values)

# ===== Step 4: 合併結果並輸出 =====
print("\n[4/4] 輸出結果...")

# 合併到原始統計區
result = stat_areas.merge(risk_df, on='U_ID', how='left')

# 填補缺失值
result['liq_risk'] = result['liq_risk'].fillna(0)

# 轉換到 WGS84
result = result.to_crs('EPSG:4326')

# 儲存結果
result.to_file(OUTPUT_PATH, driver='GeoJSON', encoding='utf-8')
print(f"  ✓ 結果已儲存至: {OUTPUT_PATH}")

# ===== 統計摘要 =====
print("\n" + "=" * 60)
print("分析結果摘要")
print("=" * 60)

print(f"\n【液化風險值 (liq_risk) 統計】")
print(f"  範圍: 0 (無風險) ~ 1 (高風險)")
print(f"  平均值: {result['liq_risk'].mean():.3f}")
print(f"  中位數: {result['liq_risk'].median():.3f}")
print(f"  標準差: {result['liq_risk'].std():.3f}")
print(f"  最大值: {result['liq_risk'].max():.3f}")


print(f"\n【覆蓋率門檻篩選結果】")
print(f"  覆蓋率門檻: {OVERLAP_THRESHOLD*100:.0f}%")
filtered_out = len(result[(result['overlap_ratio'] > 0) & (result['overlap_ratio'] < OVERLAP_THRESHOLD)])
print(f"  因覆蓋率不足而排除: {filtered_out} 個統計區")
included = len(result[result['overlap_ratio'] >= OVERLAP_THRESHOLD])
print(f"  符合門檻納入計算: {included} 個統計區")


print(f"\n【風險分級】")
high = len(result[result['liq_risk'] >= 0.67])
medium = len(result[(result['liq_risk'] >= 0.33) & (result['liq_risk'] < 0.67)])
low = len(result[(result['liq_risk'] > 0) & (result['liq_risk'] < 0.33)])
none = len(result[result['liq_risk'] == 0])

print(f"  高風險 (≥0.67): {high} 個")
print(f"  中風險 (0.33-0.67): {medium} 個")
print(f"  低風險 (0-0.33): {low} 個")
print(f"  無資料: {none} 個")

print(f"\n【風險最高的前10個統計區】")
top_risk = result.nlargest(10, 'liq_risk')[['U_ID', 'TOWN', 'liq_risk']]
print(top_risk.to_string(index=False))

# 輸出 CSV
csv_path = OUTPUT_PATH.replace('.geojson', '.csv')
result_csv = result.drop(columns=['geometry'])
result_csv.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"\n統計表格已儲存至: {csv_path}")

print("\n" + "=" * 60)
print("✅ 分析完成!")
print("📊 輸出欄位: liq_risk (範圍 0-1,可直接作為權重)")
print("=" * 60)

統計區液化風險值計算 (面積加權平均法)

[1/4] 讀取資料...
  ✓ 液化潛勢網格: 136 筆
  ✓ 最小統計區: 11490 筆
  使用欄位: class
  液化等級: ['1' '2' '3']

[2/4] 座標系統處理...
  ✓ 已統一座標系統

[3/4] 計算風險值...


/var/folders/cs/s9sl30ms3yqc42bnr0xh29pr0000gn/T/ipykernel_36479/3040062635.py:74: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlay['intersect_area'] = overlay.geometry.area
/var/folders/cs/s9sl30ms3yqc42bnr0xh29pr0000gn/T/ipykernel_36479/3040062635.py:93: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ].geometry.area.iloc[0]
/var/folders/cs/s9sl30ms3yqc42bnr0xh29pr0000gn/T/ipykernel_36479/3040062635.py:93: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ].geometry.area.iloc[0]
/var/folders/cs/s9sl30ms3yqc42bnr0xh29pr0000gn/T/ipykernel_36479/3040062635.py:93: UserWarning: Geometry


[4/4] 輸出結果...
  ✓ 結果已儲存至: taipei_liquefaction_risk.geojson

分析結果摘要

【液化風險值 (liq_risk) 統計】
  範圍: 0 (無風險) ~ 1 (高風險)
  平均值: 0.298
  中位數: 0.330
  標準差: 0.220
  最大值: 0.670

【覆蓋率門檻篩選結果】
  覆蓋率門檻: 50%
  因覆蓋率不足而排除: 489 個統計區
  符合門檻納入計算: 0 個統計區

【風險分級】
  高風險 (≥0.67): 1900 個
  中風險 (0.33-0.67): 4902 個
  低風險 (0-0.33): 3374 個
  無資料: 1314 個

【風險最高的前10個統計區】
  U_ID TOWN  liq_risk
1973.0  士林區      0.67
1974.0  士林區      0.67
2154.0  士林區      0.67
2157.0  士林區      0.67
2158.0  士林區      0.67
2159.0  士林區      0.67
2161.0  士林區      0.67
2162.0  士林區      0.67
2163.0  士林區      0.67
2164.0  士林區      0.67

統計表格已儲存至: taipei_liquefaction_risk.csv

✅ 分析完成!
📊 輸出欄位: liq_risk (範圍 0-1,可直接作為權重)


In [ ]:
import geopandas as gpd
import pandas as pd
from collections import Counter
import numpy as np

# ===== 設定參數 =====
LIQUEFACTION_PATH = 'TPLiquid_84.GEOJSON'  # 液化潛勢資料
STAT_AREA_PATH = 'STAT/113年12月臺北市統計區人口統計_最小統計區/tp_最小統計區.geojson'  # 最小統計區
OUTPUT_PATH = 'taipei_liquefaction_risk.geojson'
OVERLAP_THRESHOLD = 0.5

# 液化等級對應數值 (根據你的資料調整)
RISK_VALUES = {
    '1':0.67,
    '2': 0.33, 
    '3': 0.08,
    '4': 0
}

# '低潛勢區': 2.5/30 ≈ 0.08,   # PL中位數 2.5
# '中潛勢區': 10/30 ≈ 0.33,    # PL中位數 10
# '高潛勢區': 20/30 ≈ 0.67     # PL假設值 20

print("=" * 60)
print("統計區液化風險值計算 (面積加權平均法)")
print("=" * 60)

# ===== Step 1: 讀取資料 =====
print("\n[1/4] 讀取資料...")
liquefaction = gpd.read_file(LIQUEFACTION_PATH)
stat_areas = gpd.read_file(STAT_AREA_PATH)

print(f"  ✓ 液化潛勢網格: {len(liquefaction)} 筆")
print(f"  ✓ 最小統計區: {len(stat_areas)} 筆")

# 自動偵測液化等級欄位
risk_column = None
for col in liquefaction.columns:
    if 'class' in col or 'risk' in col.lower() or '等級' in col:
        risk_column = col
        break

if risk_column is None:
    print("\n  可用欄位:", liquefaction.columns.tolist())
    risk_column = input("  請輸入液化等級欄位名稱: ")

print(f"  使用欄位: {risk_column}")
print(f"  液化等級: {liquefaction[risk_column].unique()}")

# ===== Step 2: 座標系統統一 =====
print("\n[2/4] 座標系統處理...")

# 統一座標系統
if liquefaction.crs != stat_areas.crs:
    liquefaction = liquefaction.to_crs(stat_areas.crs)

# 轉換至 wgs84 計算面積
stat_areas_meter = stat_areas.to_crs('EPSG:4326')
liquefaction_meter = liquefaction.to_crs('EPSG:4326')

print(f"  ✓ 已統一座標系統")

# ===== Step 3: 空間疊合與計算 =====
print("\n[3/4] 計算風險值...")

# 空間疊合
overlay = gpd.overlay(
    stat_areas_meter,
    liquefaction_meter,
    how='intersection'
)

# 計算交集面積
overlay['intersect_area'] = overlay.geometry.area

# 計算每個統計區的加權平均風險值
risk_values = []

for stat_id in stat_areas_meter['U_ID'].unique():
    stat_overlay = overlay[overlay['U_ID'] == stat_id]
    
    if len(stat_overlay) == 0:
        # 無液化資料
        risk_values.append({
            'U_ID': stat_id,
            'liq_risk': 0
        })
        continue

    # 計算統計區總面積
    stat_area_total = stat_areas_meter[
        stat_areas_meter['U_ID'] == stat_id
    ].geometry.area.iloc[0]
    
    # 計算液化區覆蓋該統計區的面積
    overlap_area = stat_overlay['intersect_area'].sum()
    
    # 計算覆蓋率
    overlap_ratio = overlap_area / stat_area_total
    
    # === 覆蓋率門檻判斷 ===
    if overlap_ratio < OVERLAP_THRESHOLD:
        # 覆蓋率不足,視為邊緣效應,風險設為 0
        risk_values.append({
            'U_ID': stat_id,
            'liq_risk': 0,
            'overlap_ratio': round(overlap_ratio, 3)
        })
        continue

    # 計算總面積
    total_area = stat_overlay['intersect_area'].sum()
    
    # 面積加權平均
    weighted_sum = 0
    for _, row in stat_overlay.iterrows():
        risk_val = RISK_VALUES.get(row[risk_column], 0)
        weight = row['intersect_area'] / total_area
        weighted_sum += risk_val * weight
    
    # 標準化到 0-1
    risk_normalized = weighted_sum 
    
    risk_values.append({
        'U_ID': stat_id,
        'liq_risk': round(risk_normalized, 3)
    })

# 轉換為 DataFrame
risk_df = pd.DataFrame(risk_values)

# ===== Step 4: 合併結果並輸出 =====
print("\n[4/4] 輸出結果...")

# 合併到原始統計區
result = stat_areas.merge(risk_df, on='U_ID', how='left')

# 填補缺失值
result['liq_risk'] = result['liq_risk'].fillna(0)

# 轉換到 WGS84
result = result.to_crs('EPSG:4326')

# 儲存結果
result.to_file(OUTPUT_PATH, driver='GeoJSON', encoding='utf-8')
print(f"  ✓ 結果已儲存至: {OUTPUT_PATH}")

# ===== 統計摘要 =====
print("\n" + "=" * 60)
print("分析結果摘要")
print("=" * 60)

print(f"\n【液化風險值 (liq_risk) 統計】")
print(f"  範圍: 0 (無風險) ~ 1 (高風險)")
print(f"  平均值: {result['liq_risk'].mean():.3f}")
print(f"  中位數: {result['liq_risk'].median():.3f}")
print(f"  標準差: {result['liq_risk'].std():.3f}")
print(f"  最大值: {result['liq_risk'].max():.3f}")


print(f"\n【覆蓋率門檻篩選結果】")
print(f"  覆蓋率門檻: {OVERLAP_THRESHOLD*100:.0f}%")
filtered_out = len(result[(result['overlap_ratio'] > 0) & (result['overlap_ratio'] < OVERLAP_THRESHOLD)])
print(f"  因覆蓋率不足而排除: {filtered_out} 個統計區")
included = len(result[result['overlap_ratio'] >= OVERLAP_THRESHOLD])
print(f"  符合門檻納入計算: {included} 個統計區")


print(f"\n【風險分級】")
high = len(result[result['liq_risk'] >= 0.67])
medium = len(result[(result['liq_risk'] >= 0.33) & (result['liq_risk'] < 0.67)])
low = len(result[(result['liq_risk'] > 0) & (result['liq_risk'] < 0.33)])
none = len(result[result['liq_risk'] == 0])

print(f"  高風險 (≥0.67): {high} 個")
print(f"  中風險 (0.33-0.67): {medium} 個")
print(f"  低風險 (0-0.33): {low} 個")
print(f"  無資料: {none} 個")

print(f"\n【風險最高的前10個統計區】")
top_risk = result.nlargest(10, 'liq_risk')[['U_ID', 'TOWN', 'liq_risk']]
print(top_risk.to_string(index=False))

# 輸出 CSV
csv_path = OUTPUT_PATH.replace('.geojson', '.csv')
result_csv = result.drop(columns=['geometry'])
result_csv.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"\n統計表格已儲存至: {csv_path}")

print("\n" + "=" * 60)
print("✅ 分析完成!")
print("📊 輸出欄位: liq_risk (範圍 0-1,可直接作為權重)")
print("=" * 60)